In [ ]:
# import vertexai
from vertexai.preview.generative_models import GenerativeModel
import vertexai

# Initialize Vertex AI (must set project + region)
vertexai.init(project="qwiklabs-gcp-00-711e7fbdb4c8", location="us-central1")

# Load Gemini 2.5 Pro model
model = GenerativeModel("gemini-2.5-pro")

# post generator
def snarkPost(prompt):
  response = model.generate_content(
    """Context: You write social media posts for government announcements like weather emergencies, holidays, school closings, etc. You like to be a little snarky.

    1. Keep your posts below 150 characters
    2. Include the hashtag #We<3LocalGovernment at the end of every tweet

  Input: Write me a post letting people know there is an incoming tornado.
  Output: Better watch out, major tornado incoming, or don't, I can't make you do anything you don't want to.

  Input: {0}
  Output: """.format(prompt)
    )
  return response.text.strip()

# post checker
def does_post_follow_rules(tweet):
  response = model.generate_content(
    """Does the post follow the following rules:

    1. Keep your posts below 150 characters
    2. Include the hashtag #We<3LocalGovernment at the end of every tweet
    3. Is a little Snarky

    Only return Yes or No

    Tweet: {0}
    Output: """.format(tweet)
    )
  return response.text.strip()

# post unit test
import unittest

class TestPostRules(unittest.TestCase):

    def test_post_results_1(self):
      generated_post = snarkPost("Write a tweet about our Thanksgiving Day Special")
      correct = does_post_follow_rules(generated_post)
      self.assertEqual(correct, "Yes")

    def test_does_not_follow_rules(self):
      generated_post = "This is a run-on sentence, it has no tone to it, it is very long and probabaly written poorly, it also is only about itself and no other category, sometimes it likes to keep typing just to add more space to the sentence it exists as."
      correct = does_post_follow_rules(generated_post)
      self.assertEqual(correct, "No")


In [ ]:
# run test
postGenerated = snarkPost('Write a post about wild boars')
postGenerated_safe = postGenerated.replace("'", "\\'")
print(postGenerated_safe)

followedOrders = does_post_follow_rules(postGenerated_safe)
print(followedOrders)


In [ ]:
# question classifier
def classifyQuestionType(prompt):
  response = model.generate_content(
    """Context: You look at questions and categorize them as
    Employment, General Information, Emergency Services, or Tax Related.

  Output only Employment, General Information, Emergency Services, or Tax Related

  Input: Why are my taxes so high?
  Output: Tax Related

  Message: {0}.
  Output: """.format(prompt)
  )
  return response.text.strip()

# question unit test
import unittest # get needed library

class TestCategorization(unittest.TestCase):
  def test_getClassification(self):
    response = classifyQuestionType("Why are my pancakes cold?")
    self.assertEqual(response, "General Information")

  def test_getClassification2(self):
    response = classifyQuestionType("Where can I find job postings for train conductors?")
    self.assertEqual(response, "Employment")

  def test_getClassification3(self):
    response = classifyQuestionType("Why are my taxes so low?")
    self.assertEqual(response, "Tax Related")

  def test_getClassification4(self):
    response = classifyQuestionType("Why am I feeling light-headed and my left arm hurts?")
    self.assertEqual(response, "Emergency Services")

In [ ]:
# RUN UNIT TESTS

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
#run test
print(classifyQuestionType("How deep is the Nile River?"))

In [ ]:
# Import the pandas library
import pandas as pd
import datetime

# !pip install --upgrade google-cloud-aiplatform
from google.cloud.aiplatform import init
# from google.cloud.aiplatform.evaluation import EvalTask
from vertexai.evaluation import EvalTask

# evaluation data
classify_outputs = [
    {
        "question": "Why are my pancakes cold?",
        "classifcaiton": "General Information"
    },
    {
        "question": "Where can I find job postings for train conductors?",
        "classifcaiton": "Employment"
    },
    {
        "question": "Why are my taxes so low?",
        "classifcaiton": "Tax Related"
    },
    {
        "question": "Why am I feeling light-headed and my left arm hurts?",
        "classifcaiton": "Emergency Services"
    },
    {
        "question": "How do I get a promotion?",
        "classifcaiton": "Employment"
    },
    {
        "question": "Can I lower my tax burden by moving money from my 401k into a ROTH IRA?",
        "classifcaiton": "Tax Related"
    },
    {
        "question": "Ouch my hand hurts?",
        "classifcaiton": "Emergency Services"
    },
    {
        "question": "How deep is the Nile River?",
        "classifcaiton": "General Information"
    }
]

prompt = "Context: You look at questions and categorize them as Employment, General Information, Emergency Services, or Tax Related. Output only Employment, General Information, Emergency Services, or Tax Related"
contexts = [str(record) for record in classify_outputs]
full_prompts = [prompt + str(record) for record in classify_outputs]


# CREATE EVALUATION DATASET
eval_dataset = pd.DataFrame(
  {
    "content": full_prompts,
    "instruction": full_prompts,
    "context": contexts,
    "prompt": prompt
  }
)


# CREATE EVALUATION TASK
qa_eval_task = EvalTask(
  dataset=eval_dataset,
  metrics=["fulfillment", "groundedness"],
  experiment="question-classification-generation",
)

# Load Gemini 2.5 Pro model
model = GenerativeModel("projects/qwiklabs-gcp-00-711e7fbdb4c8/locations/us-central1/models/gemini-2.5-pro")


# RUN EVALUATION
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
result = qa_eval_task.evaluate(
    model=model,
    experiment_run_name=f"apartment-listing-gen-{run_ts}"
  )

evaluation_results = []
evaluation_results.append(result)


# DISPLAY RESULTS
display_eval_report((("Eval Result", result.summary_metrics, result.metrics_table)))
